In [38]:
from __future__ import print_function
from __future__ import division
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import re
import csv
from datetime import datetime
from tensorboardX import SummaryWriter
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print(torch.cuda.get_device_name(0))

PyTorch Version:  1.5.0
Torchvision Version:  0.5.0a0+0903a9b
GeForce GTX 1060


In [39]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "/home/mello/dev/enginx/Dataset/"
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "squeezenet"

# Number of classes in the dataset
num_classes = 9

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Number of epochs to train for
num_epochs = 20

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

#Path to the checkpoint directory
chk_dir = '/home/mello/.cache/torch/checkpoint/'

#Setting the model_history, starting epoch and the i_resume flag as global
global model_hist, start_epoch, is_resume
model_hist = {'train':[], 'test':[]}
start_epoch = 0
is_resume = True

#Path to the saved checkpoint from where to start the training
saved_path = '/home/mello/.cache/torch/checkpoint/saved_squeezenet_195_train_0.90211_test_0.84896_2020-05-06_15-32-32.pth'

## Image Name

In [40]:
def summ_image(path, text, n_iter, class_i):
    %matplotlib inline
    im = plt.imread(path)    
    writer.add_image(str(class_i), im.transpose((2,0,1)))
    plt.close()

### Train Model

In [41]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False, save_interval = 5):
    since = time.time()

    val_acc_history = []
    
    best_acc = 0.0
    
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train','test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels, paths in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_acc = float(epoch_acc.cpu().numpy())

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            model_hist[phase].append(( start_epoch + epoch, epoch_loss, epoch_acc)) ## continue from saved epoch
            
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
            if phase == 'test':
                val_acc_history.append(epoch_acc)
                
            
            if epoch%save_interval == 0 and phase == 'test':
                save_checkpoint(os.path.join(chk_dir, 'saved'),
                                   metric_dict={
                                       'epoch': model_hist['train'][-1][0],
                                       'type': model_name,
                                       'train_acc': model_hist['train'][-1][2],
                                       'test_acc': model_hist['test'][-1][2],
                                       'model_dict': model_ft.state_dict()
                                   })
            
            if phase == 'test':
                writer.add_scalars('data/scalars', {
                'train_loss': model_hist['train'][-1][1],
                'train_acc': model_hist['train'][-1][2],
                'test_acc': model_hist['test'][-1][2]
                })                    
            
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return model, val_acc_history

## Predict Model

In [42]:
def predict_model(model, dataloaders, criterion, optimizer, num_epochs=1, is_inception=False, save_interval = 5):
    since = time.time()

    
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['test']:
            model.eval()

            running_loss = 0.0
            running_corrects = 0
            for inputs, labels, paths in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)


                if phase == 'test':
                    l = int(preds.cpu().numpy())
                    summ_image(paths[0], test_class_dict[l], epoch, test_class_dict[l])
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                time_elapsed = time.time() - since
                print('Testing complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_acc = float(epoch_acc.cpu().numpy())

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            model_hist[phase].append(( start_epoch + epoch, epoch_loss, epoch_acc)) ## continue from saved epoch
            
            if phase == 'test':
                writer.add_scalars('test/scalars', {
                'test_acc': model_hist['test'][-1][2]
                }) 
        
        print()

    time_elapsed = time.time() - since
    print('Testing complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return model

In [43]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

### Initialize Model

In [44]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG16_bn
        """
        model_ft = models.vgg16_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)


In [45]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation((-30, 30)),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

# Create training and validation datasets
image_datasets = {x: ImageFolderWithPaths(os.path.join(data_dir, x, 'Root'), data_transforms[x]) for x in ['train'
                                                                                                           , 'test'
                                                                                                                ]}
# Create training and validation dataloaders
trainloader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(image_datasets['test'], batch_size=1, shuffle=True, num_workers=4)
test_class_dict = {value: key for key, value in testloader.dataset.class_to_idx.items()}
dataloaders_dict = {'train': trainloader, 'test': testloader}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [46]:
image_datasets

{'train': Dataset ImageFolderWithPaths
     Number of datapoints: 3800
     Root location: /home/mello/dev/enginx/Dataset/train/Root
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                RandomRotation(degrees=(-30, 30), resample=False, expand=False)
                RandomVerticalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'test': Dataset ImageFolderWithPaths
     Number of datapoints: 192
     Root location: /home/mello/dev/enginx/Dataset/test/Root
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [47]:
dummy = trainloader.dataset.class_to_idx
# # uncomment below for writing the dataset classes to a csv file
# with open('classes_9.csv', 'w', newline="") as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in dummy.items():
#         writer.writerow([key, value])
dummy

{'Bittergourd': 0,
 'Cabbage': 1,
 'Garlic': 2,
 'Ginger': 3,
 'Onion': 4,
 'Pepper': 5,
 'Potato': 6,
 'Pumpkin': 7,
 'Tomato': 8}

## Saver Function

In [48]:
date_pat = re.compile(
            "^(\d{4}-\d{2}-\d{2})\s(\d{2}:\d{2}:\d{2})?")
def get_datetime():  # generates date-time for logging
    match = date_pat.search(str(datetime.now()))
    date, time = match.group(1), match.group(2)
    return date, time

def save_checkpoint(path=None, metric_dict=None, mode='train'):

    date, time = get_datetime()
    time = time.replace(':', '-')
    print(date, time)
    chk_name = '{}_{}_{}_train_{}_test_{}_{}_{}.pth'.format(
        path,
        metric_dict['type'],
        metric_dict['epoch'],
        round(metric_dict['train_acc'], 5),
        round(metric_dict['test_acc'], 5),
        date,
        time
    )
    torch.save(metric_dict, chk_name)


### Load old model / Init new model

In [49]:
chk_name = saved_path
if not os.path.isfile(saved_path):
    chk_path = os.path.join(chk_dir, chk_name)
else:
    chk_path = saved_path
    
if is_resume:
    loaded =  torch.load(chk_path)
    [print(x, ": ", loaded[x]) for x in ['epoch', 'train_acc', 'test_acc']];
    model_ft.load_state_dict(loaded['model_dict'])
    start_epoch = loaded['epoch']

epoch :  195
train_acc :  0.9021052631578947
test_acc :  0.8489583333333333


In [50]:
print(device)
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

cuda:0
Params to learn:
	 classifier.1.weight
	 classifier.1.bias


In [51]:
criterion = nn.CrossEntropyLoss()

writer=SummaryWriter()

# model_ft, m_hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))
model_ft = predict_model(model_ft, dataloaders_dict, criterion, optimizer_ft, is_inception=(model_name=="inception"))
writer.close()

Epoch 0/0
----------
Testing complete in 0m 2s
Testing complete in 0m 2s
Testing complete in 0m 2s
Testing complete in 0m 2s
Testing complete in 0m 2s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 4s
Testing complete in 0m 5s
Testing complete in 0m 5s
Testing complete in 0m 5s
Testing complete in 0m 5s
Testing complete in 0m 5s
Testing complete 